<a href="https://colab.research.google.com/github/enesdouaydin/HER-SEYDEN-BIRAZ-BIRAZ/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `Qwen3-4B-Base` into a reasoning model via GRPO by using OpenR1's Math dataset.

We first pre fine-tune the model to make GRPO skip trying to match formatting - this speeds GRPO up.

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-11 07:11:22 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-11 07:11:22 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.53.1. vLLM: 0.8.5.post1.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-4B-Base with actual GPU utilization = 69.29%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 22.16 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.7.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### GRPO chat template
Since we're using a base model, we should set a chat template. You can make your own chat template as well!
1. DeepSeek uses `<think>` and `</think>`, but this is **not** necessary - you can customize it however you like!
2. A `system_prompt` is recommended to at least guide the model's responses.

In [4]:
reasoning_start = "<start_working_out>" # Acts as <think>
reasoning_end   = "<end_working_out>"   # Acts as </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

We create a simple chat template below. Notice `add_generation_prompt` includes prepending `<start_working_out>` to guide the model to start its reasoning process.

In [5]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

Let's see how our chat template behaves on an example:

In [6]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|endoftext|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|endoftext|>What is 2+2?<start_working_out>"

### Pre fine-tuning for formatting
We now use a subset of NVIDIA's [Open Math Reasoning dataset](https://huggingface.co/datasets/nvidia/OpenMathReasoning) which was filtered to only include high quality DeepSeek R1 traces.

We'll only filter ~59 or so examples to first "prime" / pre fine-tune the model to understand our custom GRPO formatting.

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
input_csv_path = "/content/drive/MyDrive/turkish_law_dataset.csv" # Buraya dosyanızın tam yolunu yapıştırın

In [8]:
import pandas as pd
import numpy as np
import json

# ---- AYARLAR ----
# 1. Girdi olarak kullanılacak CSV dosyanızın adı
input_csv_path = "/content/drive/MyDrive/turkish_law_dataset.csv" # Buraya dosyanızın tam yolunu yapıştırın
# 2. Çıktı olarak oluşturulacak .jsonl dosyasının adı
output_jsonl_path = 'hukuk_veriseti_hazir.jsonl'

# 3. CSV dosyanızdaki ilgili sütunların adları
SORU_SUTUNU = 'soru'
CEVAP_SUTUNU = 'cevap'
# Not: Eğer context sütununu da kullanmak isterseniz, onu da buraya ekleyebilirsiniz.
CONTEXT_SUTUNU = 'context'

# 4. Modele verilecek olan sistem talimatı (kimlik)
system_prompt = "Sen, Türkiye Cumhuriyeti kanunları, yönetmelikleri ve hukuki kavramlar hakkında bilgi veren bir yapay zeka asistanısın. Görevin, sorulan sorulara ilgili kanun maddelerine ve yasal ilkelere dayanarak, doğru, objektif ve tarafsız bilgiler sunmaktır. Kesinlikle kendi yorumunu katmamalı veya hukuki tavsiye vermemelisin."
# ---- AYARLAR BİTTİ ----

print("Veri işleme süreci başlatılıyor...")

# Adım 1: CSV dosyasını oku
# Orijinal kod Hugging Face'ten yüklüyordu, biz yerel CSV'den okuyoruz.
try:
    dataset = pd.read_csv(input_csv_path)
    print(f"'{input_csv_path}' başarıyla okundu. {len(dataset)} satır bulundu.")
except FileNotFoundError:
    print(f"HATA: '{input_csv_path}' dosyası bulunamadı. Lütfen dosya adını ve konumunu kontrol edin.")
    exit()


Veri işleme süreci başlatılıyor...
'/content/drive/MyDrive/turkish_law_dataset.csv' başarıyla okundu. 13954 satır bulundu.


We have to format the dataset to follow our GRPO style formatting:

In [9]:
def format_dataset(row):
    # Sütunlardan verileri al
    soru = row[SORU_SUTUNU]
    cevap = row[CEVAP_SUTUNU]

    # Eğer 'context' sütunu varsa ve boş değilse, soruya ekle
    # Bu, modele soruya cevap verirken ek bilgi sağlar.
    final_user_prompt = str(soru)
    if CONTEXT_SUTUNU in row and pd.notna(row[CONTEXT_SUTUNU]):
        context = row[CONTEXT_SUTUNU]
        final_user_prompt = f"Bağlam: {context}\n\nSoru: {soru}"

    # Unsloth örneğindeki gibi, 'assistant' cevabını oluşturuyoruz.
    # Bizim durumumuzda bu, doğrudan 'cevap' sütunundaki metindir.
    # Karmaşık <think> etiketlerine veya özel token'lara ihtiyacımız yok.
    final_assistant_response = str(cevap)

    # Nihai mesaj listesini oluştur
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : final_user_prompt.strip()},
        {"role" : "assistant", "content" : final_assistant_response.strip()},
    ]

# Adım 4: Fonksiyonu tüm veri setine uygula
print("Her bir satır, fine-tuning formatına dönüştürülüyor...")
# Önce boş olabilecek satırları temizleyelim
dataset.dropna(subset=[SORU_SUTUNU, CEVAP_SUTUNU], inplace=True)
# Fonksiyonu her satır için uygula ve yeni bir 'Messages' sütunu oluştur
dataset['Messages'] = dataset.apply(format_dataset, axis=1)

# Adım 5: Sonuçları JSONL dosyasına yaz
print(f"Sonuçlar '{output_jsonl_path}' dosyasına yazılıyor...")
with open(output_jsonl_path, 'w', encoding='utf-8') as f:
    for messages_list in dataset['Messages']:
        # Her bir mesaj listesini JSON formatında bir satır olarak yaz
        f.write(json.dumps({"messages": messages_list}, ensure_ascii=False) + '\n')

print("\nİşlem başarıyla tamamlandı!")
print(f"Veri setiniz, fine-tuning için hazır hale getirilerek '{output_jsonl_path}' dosyasına kaydedildi.")

Her bir satır, fine-tuning formatına dönüştürülüyor...
Sonuçlar 'hukuk_veriseti_hazir.jsonl' dosyasına yazılıyor...

İşlem başarıyla tamamlandı!
Veri setiniz, fine-tuning için hazır hale getirilerek 'hukuk_veriseti_hazir.jsonl' dosyasına kaydedildi.


Check to see if it worked:

In [10]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

'Sen, Türkiye Cumhuriyeti kanunları, yönetmelikleri ve hukuki kavramlar hakkında bilgi veren bir yapay zeka asistanısın. Görevin, sorulan sorulara ilgili kanun maddelerine ve yasal ilkelere dayanarak, doğru, objektif ve tarafsız bilgiler sunmaktır. Kesinlikle kendi yorumunu katmamalı veya hukuki tavsiye vermemelisin.<|endoftext|>Bağlam: BAŞLANGIÇ [5]\n\n \n\nTürk Vatanı ve Milletinin ebedi varlığını ve Yüce Türk Devletinin bölünmez bütünlüğünü belirleyen bu Anayasa, Türkiye Cumhuriyetinin kurucusu, ölümsüz önder ve eşsiz kahraman Atatürk’ün belirlediği milliyetçilik anlayışı ve onun inkılap ve ilkeleri doğrultusunda;\n\nDünya milletleri ailesinin eşit haklara sahip şerefli bir üyesi olarak, Türkiye Cumhuriyetinin ebedi varlığı, refahı, maddi ve manevi mutluluğu ile çağdaş medeniyet düzeyine ulaşma azmi yönünde;\n\nMillet iradesinin mutlak üstünlüğü, egemenliğin kayıtsız şartsız Türk Milletine ait olduğu ve bunu millet adına kullanmaya yetkili kılınan hiçbir kişi ve kuruluşun, bu Anayas

Let's truncate the pre fine-tuning dataset to `max_seq_length/2` since we don't want too long reasoning traces.

Note this might take 2 minutes!

In [11]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

(2029, 8)

We then tokenize the messages and convert it to a Hugging Face compatible dataset format:

In [12]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['soru', 'cevap', 'veri türü', 'kaynak', 'context', 'Score', 'Messages', 'N', 'text', '__index_level_0__'],
    num_rows: 2029
})

Let's now pre fine-tune the model so it follows our custom GRPO formatting!

In [20]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        # ----> BU SATIR EN ÖNEMLİSİ <----
        output_dir = "turk-hukuk-modeli", # Modelin kaydedileceği klasörün adı

        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/2029 [00:00<?, ? examples/s]

In [21]:
!ls -l


total 111188
drwx------ 5 root root      4096 Jul 11 07:19 drive
drwxr-xr-x 4 root root      4096 Jul 11 07:18 huggingface_tokenizers_cache
-rw-r--r-- 1 root root 113823324 Jul 11 07:19 hukuk_veriseti_hazir.jsonl
drwxr-xr-x 1 root root      4096 Jul  9 21:17 sample_data
drwxr-xr-x 2 root root      4096 Jul 11 07:32 turk-hukuk-modeli
drwxr-xr-x 3 root root      4096 Jul 11 07:11 unsloth_compiled_cache
drwxr-xr-x 2 root root      4096 Jul 11 07:20 unsloth_training_checkpoints
-rw-r--r-- 1 root root      2403 Jul 11 07:09 vllm_requirements.txt


In [22]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,029 | Num Epochs = 1 | Total steps = 2,029
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)


Step,Training Loss
5,0.126100
10,0.078800
15,0.087600
20,0.081200
25,0.102000
30,0.110700
35,0.079200
40,0.128100
45,0.077100
50,0.120500


TrainOutput(global_step=2029, training_loss=0.08699104190607151, metrics={'train_runtime': 1355.9987, 'train_samples_per_second': 1.496, 'train_steps_per_second': 1.496, 'total_flos': 3.6947627801088e+16, 'train_loss': 0.08699104190607151})

kayıt asaması


In [41]:
# Drive'da 'modellerim' adında bir klasör oluşturduğunuzdan emin olun
!mkdir -p "/content/drive/MyDrive/modellerim"

# Kopyalama işlemi
# Kaynak: /content/turk-hukuk-modeli/
# Hedef: /content/drive/MyDrive/modellerim/turk-hukuk-modeli/
!cp -r /content/turk-hukuk-modeli "/content/drive/MyDrive/modellerim/"

print("Modeliniz, Google Drive'daki 'modellerim' klasörüne başarıyla kopyalandı.")

Modeliniz, Google Drive'daki 'modellerim' klasörüne başarıyla kopyalandı.


In [23]:
from huggingface_hub import notebook_login
notebook_login()

In [24]:
# "kullanici-adiniz" ve "model-adiniz" kısımlarını kendinize göre değiştirin
# Örnek: "dou/turk-hukuk-asistani-v1"
trainer.push_to_hub("GOOLGMAN/qwen3_4b_grpo_finetune_hukuk_modeli_v1")

  0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/264M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/GOOLGMAN/turk-hukuk-modeli/commit/931bd6ac8992dcb21d655d6bbde9cb4610184dba', commit_message='GOOLGMAN/qwen3_4b_grpo_finetune_hukuk_modeli_v1', commit_description='', oid='931bd6ac8992dcb21d655d6bbde9cb4610184dba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/GOOLGMAN/turk-hukuk-modeli', endpoint='https://huggingface.co', repo_type='model', repo_id='GOOLGMAN/turk-hukuk-modeli'), pr_revision=None, pr_num=None)

In [26]:
import os

# ---- AYARLAR ----
# 1. Drive'a kopyaladığınız model klasörünün adı
model_klasor_adi = "turk-hukuk-modeli"

# 2. Modelinizin Drive'daki ana klasörünün yolu
drive_ana_klasoru = "/content/drive/MyDrive/modellerim"
# ---- AYARLAR BİTTİ ----


# Kaynak klasörün tam yolunu oluşturuyoruz (Zip'lenecek klasör)
kaynak_klasor_yolu = os.path.join(drive_ana_klasoru, model_klasor_adi)

# Oluşturulacak ZIP dosyasının tam yolunu ve adını oluşturuyoruz
zip_dosya_yolu = os.path.join(drive_ana_klasoru, f"{model_klasor_adi}.zip")


# Önce kaynak klasörün gerçekten var olup olmadığını kontrol edelim
if not os.path.isdir(kaynak_klasor_yolu):
    print(f"HATA: Kaynak klasör bulunamadı!")
    print(f"Lütfen '{kaynak_klasor_yolu}' konumunda bir klasör olduğundan emin olun.")
    print("Modeli önce Adım 1'deki gibi Drive'a kopyaladınız mı?")
else:
    print(f"Kaynak klasör bulundu: {kaynak_klasor_yolu}")
    print(f"Hedef ZIP dosyası: {zip_dosya_yolu}")
    print("Zip'leme işlemi başlatılıyor... Bu işlem klasör boyutuna göre birkaç dakika sürebilir.")

    # Colab'in terminalinde zip komutunu çalıştırıyoruz
    # -r parametresi: Klasörün içindekilerle birlikte (recursive) sıkıştırma yapar.
    # İlk yol: Oluşturulacak zip dosyasının yolu.
    # İkinci yol: Sıkıştırılacak klasörün yolu.
    !zip -r "{zip_dosya_yolu}" "{kaynak_klasor_yolu}"

    print("\nİşlem başarıyla tamamlandı!")
    print(f"Modeliniz, '{zip_dosya_yolu}' konumunda bir ZIP dosyası olarak arşivlendi.")
    print("Artık Google Drive'ınızdaki 'modellerim' klasöründen bu dosyayı indirebilirsiniz.")

Kaynak klasör bulundu: /content/drive/MyDrive/modellerim/turk-hukuk-modeli
Hedef ZIP dosyası: /content/drive/MyDrive/modellerim/turk-hukuk-modeli.zip
Zip'leme işlemi başlatılıyor... Bu işlem klasör boyutuna göre birkaç dakika sürebilir.
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/ (stored 0%)
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/README.md (deflated 43%)
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/checkpoint-500/ (stored 0%)
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/checkpoint-500/README.md (deflated 65%)
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/checkpoint-500/adapter_model.safetensors (deflated 7%)
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/checkpoint-500/adapter_config.json (deflated 57%)
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/checkpoint-500/chat_template.jinja (deflated 55%)
  adding: content/drive/MyDrive/modellerim/turk-hukuk-modeli/checkpoint-500

In [40]:
# Drive'daki model klasörünü, yine Drive'da bir zip dosyasına dönüştürün
!zip -r "/content/drive/MyDrive/modellerim/hukuk-modeli-v1.zip" "/content/drive/MyDrive/modellerim/hukuk-modeli-v1"

	zip warning: name not matched: /content/drive/MyDrive/modellerim/hukuk-modeli-v1

zip error: Nothing to do! (try: zip -r /content/drive/MyDrive/modellerim/hukuk-modeli-v1.zip . -i /content/drive/MyDrive/modellerim/hukuk-modeli-v1)


In [ ]:
from transformers import TFAutoModelForSequenceClassification

# Kayıtlı modelin yolu olarak değişkenimizi kullanıyoruz.
kayitli_model_yolu = turk-hukuk-modeli

# Modeli TensorFlow/Keras formatında yükle
tf_model = TFAutoModelForSequenceClassification.from_pretrained(
    kayitli_model_yolu,
    from_pt=True
)

# Keras formatında kaydedilecek klasör adını da dinamik olarak oluştur
keras_kayit_yolu = f"/content/drive/MyDrive/modellerim/{turk-hukuk-modeli}-keras-formatinda"
tf_model.save_pretrained(keras_kayit_yolu)

print(f"Model, Keras formatında '{keras_kayit_yolu}' klasörüne kaydedildi.")

Let's check if the model has learnt to follow the custom format:

> Blok alıntı ekle



In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_yolu = "turk-hukuk-modeli"

# Sadece CPU kullan
model = AutoModelForCausalLM.from_pretrained(model_yolu, device_map="cpu")
tokenizer = AutoTokenizer.from_pretrained(model_yolu)

device = "cpu"
model.to(device)

print("Model CPU'da yüklendi. GPU boş değilse bu tek güvenli yol.")


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 11.38 MiB is free. Process 18659 has 22.14 GiB memory in use. Of the allocated memory 21.07 GiB is allocated by PyTorch, with 57.88 MiB allocated in private pools (e.g., CUDA Graphs), and 93.42 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [31]:
def hukuki_soru_sor(soru, sistem_mesaji=None):
    """
    Fine-tune edilmiş hukuk modeline bir soru sorar ve cevabını yazdırır.
    """
    if sistem_mesaji is None:
        # Eğitimde kullandığımız standart sistem mesajı
        sistem_mesaji = "Sen, Türkiye Cumhuriyeti kanunları, yönetmelikleri ve hukuki kavramlar hakkında bilgi veren bir yapay zeka asistanısın. Görevin, sorulan sorulara ilgili kanun maddelerine ve yasal ilkelere dayanarak, doğru, objektif ve tarafsız bilgiler sunmaktır. Kesinlikle kendi yorumunu katmamalı veya hukuki tavsiye vermemelisin."

    # Mesajları modelin anlayacağı formata getiriyoruz
    mesajlar = [
        {"role": "system", "content": sistem_mesaji},
        {"role": "user", "content": soru}
    ]

    # Cevap üretmesi için özel token'ları ekleyerek girdiyi hazırlıyoruz
    prompt = tokenizer.apply_chat_template(mesajlar, tokenize=False, add_generation_prompt=True)

    # Girdiyi token'larına ayırıp GPU'ya gönderiyoruz
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    print(f"Soru: {soru}")
    print("\n--- Modelin Cevabı ---")

    # Modelden cevap üretmesini istiyoruz
    outputs = model.generate(**inputs, max_new_tokens=512, do_sample=False) # do_sample=False, sıcaklığı 0 gibi yapar

    # Üretilen token'ları tekrar metne çeviriyoruz
    cevap_tokenlari = outputs[0][inputs["input_ids"].shape[1]:] # Sadece yeni üretilen kısmı al
    cevap_metni = tokenizer.decode(cevap_tokenlari, skip_special_tokens=True)

    print(cevap_metni)
    print("----------------------\n")

In [32]:
# Test 1: Basit bir kanun maddesi sorusu
hukuki_soru_sor("Türk Ceza Kanunu'nun 81. maddesi nedir?")

# Test 2: Bir hukuki kavramın tanımı
hukuki_soru_sor("İhtiyati tedbir ne demektir ve hangi durumlarda istenir?")

# Test 3: Süre ile ilgili bir soru
hukuki_soru_sor("İşe iade davası açmak için yasal süre ne kadardır?")

# Test 4: Veri setinizde olmayan ama mantık yürütmesini gerektirebilecek bir soru
hukuki_soru_sor("Bir kiracı, kirayı 3 gün geciktirirse ev sahibi onu hemen tahliye edebilir mi?")

Soru: Türk Ceza Kanunu'nun 81. maddesi nedir?

--- Modelin Cevabı ---


AttributeError: 'Qwen3Model' object has no attribute 'max_seq_length'

In [39]:
# Canlı metin akışı için gerekli kütüphaneyi import et (genellikle hücrenin en başında yapılır)
from transformers import TextStreamer

# 1. Test için bir girdi metni hazırla
# Veri setindeki ilk diyaloğun sistem ve kullanıcı mesajlarını alıyoruz.
# 'add_generation_prompt=True' ile modelin cevap vermesi için hazırlıyoruz.
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True,
)

# Modele tam olarak ne gönderildiğini görmek için bu kısmı ekledim
print("--- Modele Gönderilen Girdi ---")
print(text)
print("---------------------------------")
print("\n--- Modelin Cevabı ---")

# 2. Modelden canlı olarak cevap üretmesini iste
# Girdiyi token'larına ayırıp modele uygun formata getiriyoruz
inputs = tokenizer(text, return_tensors="pt").to("cuda")

# Canlı yazdırma aracını oluşturuyoruz
# skip_prompt=True, sadece yeni üretilen cevabı yazdırmasını sağlar, girdiyi tekrar etmez.
streamer = TextStreamer(tokenizer, skip_prompt=True)

_ = model.generate(
    **inputs,
    temperature = 0,
    max_length=512,
    max_new_tokens = 1024, # Üretilecek maksimum yeni token sayısı
    streamer = streamer,
)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=1024) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


--- Modele Gönderilen Girdi ---
Sen, Türkiye Cumhuriyeti kanunları, yönetmelikleri ve hukuki kavramlar hakkında bilgi veren bir yapay zeka asistanısın. Görevin, sorulan sorulara ilgili kanun maddelerine ve yasal ilkelere dayanarak, doğru, objektif ve tarafsız bilgiler sunmaktır. Kesinlikle kendi yorumunu katmamalı veya hukuki tavsiye vermemelisin.<|endoftext|>Bağlam: BAŞLANGIÇ [5]

 

Türk Vatanı ve Milletinin ebedi varlığını ve Yüce Türk Devletinin bölünmez bütünlüğünü belirleyen bu Anayasa, Türkiye Cumhuriyetinin kurucusu, ölümsüz önder ve eşsiz kahraman Atatürk’ün belirlediği milliyetçilik anlayışı ve onun inkılap ve ilkeleri doğrultusunda;

Dünya milletleri ailesinin eşit haklara sahip şerefli bir üyesi olarak, Türkiye Cumhuriyetinin ebedi varlığı, refahı, maddi ve manevi mutluluğu ile çağdaş medeniyet düzeyine ulaşma azmi yönünde;

Millet iradesinin mutlak üstünlüğü, egemenliğin kayıtsız şartsız Türk Milletine ait olduğu ve bunu millet adına kullanmaya yetkili kılınan hiçbir kişi 

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 11.38 MiB is free. Process 18659 has 22.14 GiB memory in use. Of the allocated memory 21.07 GiB is allocated by PyTorch, with 57.88 MiB allocated in private pools (e.g., CUDA Graphs), and 84.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Yes it did follow the formatting! Great! Let's remove some items before the GRPO step

> Blok alıntı ekle



And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [35]:
model.save_lora("grpo_saved_lora")

AttributeError: 'Qwen3ForCausalLM' object has no attribute 'save_lora'

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Okay, so I need to find the square root of 101. Hmm, let me think. I remember that square roots of numbers between perfect squares are irrational, meaning they can't be expressed as simple fractions. Let's see, the perfect squares around 101 are 100 and 121. Since 10² is 100 and 11² is 121, the square root of 101 must be between 10 and 11. \n\nWait, but how close is it to 10 or 11? Maybe I can estimate it a bit more. Let me try multiplying 10.05 by itself. Calculating (10.05)² would be 10.05 * 10.05. Let's break that down: 10 * 10 = 100, 10*0.05=0.5, 0.05*10=0.5, and 0.05*0.05=0.0025. Adding those up: 100 + 0.5 + 0.5 + 0.0025 = 101.0025. That's a bit more than 101. So maybe 10.05 is too high. \n\nLet me try 10.04... Let's compute (10.04)². That would be 10.04 * 10.04. Breaking it down: 100 + 40*0.04=1.6 + 0.04*0.04=0.0016. So 100 + 1.6 + 0.0016 = 101.6016. Still a bit high. So maybe 10.03. Let's try (10.03)²: 100 + 30*0.03=0.9 + 0.03*0.03=0.0009. Adding gives 100 + 0.9 + 0.0009 = 100.

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
